In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

# Distribuição Zero-Inflated Poisson (ZIP) e
# Zero-Inflated Binomial Negativa (ZINB)

Exmplo da aula do MBA DSA USP/ESALQ com Prof. Luiz Fávero


### Objetivo

Analisar o comportamento de uma variável depedente em função de variáveis preditoras.<br>

**Capacidade de capturar o comportamento de inflação de zeros sem Cauda Longa**

Quantidade excessiva de Zeros na variável dependente.

Realizar o teste de Vuong (1989) para vericiar a quantiade excessiva de zeros.

### Contexto

Fisman, R.; Miguel, E. Corruption, Norms, and Legal Enforcement:<br>
Evidence from Diplomatic Parking Tickets.<br>
Journal of Political Economy, v. 15, n. 6, p. 1020-1048, 2007.<br>
https://www.journals.uchicago.edu/doi/abs/10.1086/527495

Base contendo evidências de corrupção em multas de estacionamento.

Os membros de corpo diplomático não precisam pagar multa caso estacionem em lugar proibido.<br>


- country
- violations: qtde de violações de trânsito por mês
- staff: qtde de membros do corpo diplomático
- post: antes ou depois da obrigatotriedade de multa instituído pelo prefeitura de NY
- corruption: indicador de corrupção do pais. Maior pior.

**Constructor**

violations/month ~ staff + post + corruption

ln(violations) = alpha + B1staff + B2post + B3corruption

In [2]:
#Importando a base
df_corrupcao = pd.read_csv(r'bases\corruption.csv')
df_corrupcao

,country,code,violations,staff,post,corruption
0,Angola,AGO,50,9,no,1.047506
1,Angola,AGO,1,9,yes,1.047506
2,Albania,ALB,17,3,no,0.921079
3,Albania,ALB,0,3,yes,0.921079
4,United Arab Emirates,ARE,0,3,no,-0.779468
...,...,...,...,...,...,...
293,Zaire,ZAR,0,6,yes,1.582807
294,Zambia,ZMB,37,9,no,0.562193
295,Zambia,ZMB,0,9,yes,0.562193
296,Zimbabwe,ZWE,43,14,no,0.133379


## Aplicando Modelo ZIP

In [14]:
#https://www.statsmodels.org/stable/glm.html#links
import statsmodels.api as sm

#Definindo variáveis do modelo Poisson
X = sm.add_constant(df_corrupcao.iloc[:,3:6]) #add constante para cálculo da função
y = df_corrupcao.violations

#Definindo variáveis do modelo Inflação e add constante para cálculo da função
x_infl = sm.add_constant(df_corrupcao.corruption)

#Para esse modelo é necessário transformar a variável qualitativa em Dummie
X_dum = pd.get_dummies(X, 'post', drop_first=True)


modelo_zip = sm.ZeroInflatedPoisson(endog=y, exog=X_dum, exog_infl=x_infl).fit()


Optimization terminated successfully.
         Current function value: 5.977545
         Iterations: 22
         Function evaluations: 27
         Gradient evaluations: 27


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [16]:
modelo_zip.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     ZeroInflatedPoisson Regression Results                    
===============================================================================
Dep. Variable:              violations   No. Observations:                  298
Model:             ZeroInflatedPoisson   Df Residuals:                      294
Method:                            MLE   Df Model:                            3
Date:                 Wed, 29 Mar 2023   Pseudo R-squ.:                  0.1975
Time:                         20:09:05   Log-Likelihood:                -1781.3
converged:                        True   LL-Null:                       -2219.8
Covariance Type:             nonrobust   LLR p-value:                8.828e-190
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
inflate_const         -1.6116      0.240     -6.714      0.000      -2.082      -1.141
inflate_corruption    -0.9523      0.195     -4.875      0.000      -1.335      -0.569
const                  2.4889      0.032     78.990      0.000       2.427       2.551
staff                  0.0200      0.001     16.154      0.000       0.018       0.022
corruption             0.0937      0.030      3.127      0.002       0.035       0.152
post_yes              -4.2879      0.201    -21.291      0.000      -4.683      -3.893
======================================================================================
"""

## Teste de Vuong

Verificar se a partir do modelo estimado contra o modelo de poisson possui inflação de zeros.

In [20]:
#https://stackoverflow.com/questions/75727494/how-can-i-code-vuongs-statistical-test-in-python
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import norm
from sklearn.datasets import load_breast_cancer

def vuong_test(mod1, mod2, correction=True):
    '''
    mod1, mod2 - non-nested logitstic regression fit results from statsmodels
    '''
    # number of observations and check of models
    N = mod1.nobs
    N2 = mod2.nobs
    if N != N2:
        raise ValueError('Models do not have the same number of observations')
    # extract the log-likelihood for individual points with the models
    m1 = mod1.model.loglikeobs(mod1.params)
    m2 = mod2.model.loglikeobs(mod2.params)
    # point-wise log likelihood ratio
    m = m1 - m2
    # calculate the LR statistic
    LR = np.sum(m)
    # calculate the AIC and BIC correction factors -> these go to zero when df is same between models
    AICcor = mod1.df_model - mod2.df_model
    BICcor = np.log(N)*AICcor/2
    # calculate the omega^2 term
    omega2 = np.var(m, ddof=1)
    # calculate the Z statistic with and without corrections
    Zs = np.array([LR,LR-AICcor,LR-BICcor])
    Zs /= np.sqrt(N*omega2)
    # calculate the p-value
    ps = []
    msgs = []
    for Z in Zs:
        if Z>0:
            ps.append(1 - norm.cdf(Z))
            msgs.append('model 1 preferred over model 2')
        else:
            ps.append(norm.cdf(Z))
            msgs.append('model 2 preferred over model 1')
    # share information
    print('=== Vuong Test Results ===')
    labs = ['Uncorrected']
    if AICcor!=0:
        labs += ['AIC Corrected','BIC Corrected']
    for lab,msg,p,Z in zip(labs,msgs,ps,Zs):
        print('  -> '+lab)
        print('    -> '+msg)
        print('    -> Z: '+str(Z))
        print('    -> p: '+str(p))

**Resultado da função equivale ao RAW na função em R onde p < 0,05 rejeita H0. Existe inflação de Zeros**

In [21]:

#Cria modelo Poisson
modelo_poisson = sm.Poisson(y, X_dum).fit()

#Aplica Teste
vuong_test(modelo_zip, modelo_poisson)

Optimization terminated successfully.
         Current function value: 6.952328
         Iterations 9
=== Vuong Test Results ===
  -> Uncorrected
    -> model 1 preferred over model 2
    -> Z: 2.9877590232091156
    -> p: 0.0014051552109882737


## LogLike Ratior Test

In [32]:
def llrtest (model_ref, model_test):
    
    from scipy.stats import chi2
    
    if model_ref == model_test:

        stats_chi2 = -2*(model_ref.llnull - model_test.llf)
        
        return print(f"""LogLike Modelo Nulo: {model_ref.llnull:.2f}
LogLike Modelo: {model_test.llf:.2f}
Estatistica Chi2: {stats_chi2:.2f}
P-value Chi2: {1-chi2.cdf(stats_chi2, df=model_test.df_model)}""")
            
    else:
        
        stats_chi2 = -2*(model_ref.llf - model_test.llf)
        
        return print(f"""LogLike Modelo Ref: {model_ref.llf:.2f}
LogLike Modelo: {model_test.llf:.2f} 
Estatistica Chi2: {stats_chi2:.2f}
P-value Chi2: {1-chi2.cdf(stats_chi2, df=model_test.df_model)}""")

### Teste a favor do modelo ZIP com LL estatisticamente maior do que o modelo Referência (Poisson).

In [33]:
llrtest(modelo_poisson, modelo_zip)

LogLike Modelo Ref: -2071.79
LogLike Modelo: -1781.31 
Estatistica Chi2: 580.97
P-value Chi2: 0.0


## Comparando os Modelo

In [44]:
print(f'modelo poisson: {modelo_poisson.predict([1, 23, 0.5, 0])}')
#print(f'modelo ZIP: {modelo_zip.predict([1,0.5, 1, 23, 0.5, 0])}')

modelo poisson: [17.93283965]


In [38]:
modelo_zip.params

inflate_const        -1.611649
inflate_corruption   -0.952315
const                 2.488877
staff                 0.020020
corruption            0.093722
post_yes             -4.287916
dtype: float64

In [43]:
# Cálculo manual da Previsão do Modelo ZIP com 
#Corruption 0.5
#Staff 23
#Post_yes No
(1 - (1/(1 + np.exp(-(-1.6117 - 0.9524*0.5)))))*(np.exp(2.488877 + 0.020020*23 - 4.287916*0 + 0.093722*0.5))

17.80272023251229

# Zero-Inflated Binomial Negativa (ZINB)

### Objetivo

Analisar o comportamento de uma variável depedente em função de variáveis preditoras.<br>

**Capacidade de capturar o comportamento de inflação de zeros COM Cauda Longa**

In [45]:
modelo_zinb = sm.ZeroInflatedNegativeBinomialP(y, X_dum, exog_infl=x_infl).fit()
modelo_zinb.summary()

         Current function value: 1.870015
         Iterations: 35
         Function evaluations: 39
         Gradient evaluations: 39


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                     ZeroInflatedNegativeBinomialP Regression Results                    
=========================================================================================
Dep. Variable:                        violations   No. Observations:                  298
Model:             ZeroInflatedNegativeBinomialP   Df Residuals:                      294
Method:                                      MLE   Df Model:                            3
Date:                           Wed, 29 Mar 2023   Pseudo R-squ.:                  0.1700
Time:                                   20:57:10   Log-Likelihood:                -557.26
converged:                                 False   LL-Null:                       -671.37
Covariance Type:                       nonrobust   LLR p-value:                 3.353e-49
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
inflate_const        -17.9857      9.287     -1.937      0.053     -36.187       0.216
inflate_corruption    -8.1104      4.037     -2.009      0.045     -16.023      -0.197
const                  2.0324      0.203      9.988      0.000       1.634       2.431
staff                  0.0411      0.013      3.078      0.002       0.015       0.067
corruption             0.1815      0.147      1.237      0.216      -0.106       0.469
post_yes              -4.2638      0.261    -16.328      0.000      -4.776      -3.752
alpha                  1.8560      0.218      8.518      0.000       1.429       2.283
======================================================================================
"""

In [47]:
theta = 1/modelo_zinb.params['alpha']
theta

0.538788500861243

### Teste de Vuong para verificar Ocorrência de Inflação de Zeros entre o modelo Bneg e ZINB

p_value < 0,05, existe inflação de zeros

In [51]:
modelo_bneg = sm.NegativeBinomial(y, X_dum).fit()
vuong_test(modelo_zinb, modelo_bneg)

Optimization terminated successfully.
         Current function value: 1.904031
         Iterations: 19
         Function evaluations: 23
         Gradient evaluations: 23
=== Vuong Test Results ===
  -> Uncorrected
    -> model 1 preferred over model 2
    -> Z: 1.944023255622406
    -> p: 0.02594632338431324


## LogLike Ratio Teste Para verificar melhoria do LL em relação ao Modelo Bneg

p_value < 0,05, o ganho é estatisticamente significativo

In [50]:
llrtest(modelo_bneg, modelo_zinb)

LogLike Modelo Ref: -567.40
LogLike Modelo: -557.26 
Estatistica Chi2: 20.27
P-value Chi2: 0.00014896052524815406


**O modelo Zero Inflated Negative Binomial é o que melhor captura a inflação de zeros e a cauda longa dos dados da variável dependente.** 

## Incluindo Fitted Values na Base

In [57]:
df_corrupcao['fitted_poisson'] = modelo_poisson.predict(X_dum)
df_corrupcao['fitted_bneg'] = modelo_bneg.predict(X_dum)
df_corrupcao['fitted_zip'] = modelo_zip.predict(X_dum)
df_corrupcao['fitted_zinb'] = modelo_zinb.predict(X_dum)

In [58]:
df_corrupcao

,country,code,violations,staff,post,corruption,fitted_poisson,fitted_bneg,fitted_zip,fitted_zinb
0,Angola,AGO,50,9,no,1.047506,15.919915,16.138708,14.823562,13.358543
1,Angola,AGO,1,9,yes,1.047506,0.216711,0.224608,0.203580,0.187939
2,Albania,ALB,17,3,no,0.921079,13.371720,11.987745,12.877979,10.203814
3,Albania,ALB,0,3,yes,0.921079,0.182023,0.166838,0.176860,0.143555
4,United Arab Emirates,ARE,0,3,no,-0.779468,7.477915,5.551744,8.379367,7.494531
...,...,...,...,...,...,...,...,...,...,...
293,Zaire,ZAR,0,6,yes,1.582807,0.243691,0.253817,0.207250,0.183098
294,Zambia,ZMB,37,9,no,0.562193,13.486751,12.955729,13.616058,12.232444
295,Zambia,ZMB,0,9,yes,0.562193,0.183589,0.180310,0.186996,0.172096
296,Zimbabwe,ZWE,43,14,no,0.133379,12.994217,13.033457,13.732195,13.896783


# Modelo OLS para Comparação

Nível ineficiência do modelo de Regressão Linear

In [59]:
modelo_lm = sm.OLS(y, X_dum).fit()
modelo_lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             violations   R-squared:                       0.150
Model:                            OLS   Adj. R-squared:                  0.142
Method:                 Least Squares   F-statistic:                     17.34
Date:                Wed, 29 Mar 2023   Prob (F-statistic):           2.15e-10
Time:                        21:10:53   Log-Likelihood:                -1262.9
No. Observations:                 298   AIC:                             2534.
Df Residuals:                     294   BIC:                             2549.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.7926      1.756      5.577      0.000       6.337      13.248
staff          0.2538      0.091      2.775      0.006       0.074       0.434
corruption     2.2367      1.001      2.235      0.026       0.267       4.206
post_yes     -12.6443      1.955     -6.469      0.000     -16.491      -8.797
==============================================================================
Omnibus:                      355.094   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            20158.543
Skew:                           5.312   Prob(JB):                         0.00
Kurtosis:                      41.867   Cond. No.                         38.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Teste de Verificação da Aderência dos Resíduos à Normalidade (Shapiro-Francia)

Caso os erros não sigam uma distribuição normal, o modelo não é adequado para fins preditivos. <br>
Os betas não são ajustados para que consigam fazer inferência a partir dos intervalos de confiança daqueles parâmetros e construção de cenários.

> Teste Shapiro-Francia: n >= 30

> Teste Shapiro-Wilk: n < 30


onde,

 - H0: Diferênças não são estatisticamente significantes (existe aderência à normalidade);
 - H1: Diferênças são estatisticamente significantes (não é uma normal)
 
p-value < 0,05 rejeita-se H0

In [60]:
from scipy.stats import shapiro

stat, p_value = shapiro(modelo_lm.resid)
stat , p_value

(0.5370519161224365, 2.6539385893064052e-27)

In [63]:
llrtest(modelo_zinb, modelo_lm)

LogLike Modelo Ref: -557.26
LogLike Modelo: -1262.86 
Estatistica Chi2: -1411.19
P-value Chi2: 1.0


### Transformação de Box-Cox

O modelo LM não possui resíduos aderentes à normalidade pelo teste Shapiro-Francia.<br>
Devemos transformar a variável Y em uma normal aplicando o método de Box Cox:

Antes precisamos somar 0.0001 na variável Y para transformação de Box-Cox pois zeros impedem a transformação.

In [65]:
df_corrupcao['violations1'] = df_corrupcao.violations + 0.001

In [66]:
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.boxcox.html
from scipy import stats

y_lamb, lambida_otimo = stats.boxcox(df_corrupcao.violations1)
lambida_otimo

-0.06364174393948599

O lambda é próximo de zero para garantir a forma funcional semi-logarítima para dados de contagem

In [67]:
#Inserindo a variável transformada no banco de dados
df_corrupcao['bc_violations'] = y_lamb

In [68]:
df_corrupcao

,country,code,violations,staff,post,corruption,fitted_poisson,fitted_bneg,fitted_zip,fitted_zinb,violations1,bc_violations
0,Angola,AGO,50,9,no,1.047506,15.919915,16.138708,14.823562,13.358543,50.001,3.463074
1,Angola,AGO,1,9,yes,1.047506,0.216711,0.224608,0.203580,0.187939,1.001,0.000999
2,Albania,ALB,17,3,no,0.921079,13.371720,11.987745,12.877979,10.203814,17.001,2.592518
3,Albania,ALB,0,3,yes,0.921079,0.182023,0.166838,0.176860,0.143555,0.001,-8.675434
4,United Arab Emirates,ARE,0,3,no,-0.779468,7.477915,5.551744,8.379367,7.494531,0.001,-8.675434
...,...,...,...,...,...,...,...,...,...,...,...,...
293,Zaire,ZAR,0,6,yes,1.582807,0.243691,0.253817,0.207250,0.183098,0.001,-8.675434
294,Zambia,ZMB,37,9,no,0.562193,13.486751,12.955729,13.616058,12.232444,37.001,3.226073
295,Zambia,ZMB,0,9,yes,0.562193,0.183589,0.180310,0.186996,0.172096,0.001,-8.675434
296,Zimbabwe,ZWE,43,14,no,0.133379,12.994217,13.033457,13.732195,13.896783,43.001,3.344928


In [69]:
modelo_bc = sm.OLS(df_corrupcao.bc_violations, X_dum).fit()
modelo_bc.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          bc_violations   R-squared:                       0.513
Model:                            OLS   Adj. R-squared:                  0.508
Method:                 Least Squares   F-statistic:                     103.4
Date:                Wed, 29 Mar 2023   Prob (F-statistic):           1.04e-45
Time:                        21:21:43   Log-Likelihood:                -803.49
No. Observations:                 298   AIC:                             1615.
Df Residuals:                     294   BIC:                             1630.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.8700      0.376     -2.315      0.021      -1.610      -0.130
staff          0.0431      0.020      2.203      0.028       0.005       0.082
corruption     1.1964      0.214      5.585      0.000       0.775       1.618
post_yes      -6.9798      0.418    -16.682      0.000      -7.803      -6.156
==============================================================================
Omnibus:                        7.587   Durbin-Watson:                   1.935
Prob(Omnibus):                  0.023   Jarque-Bera (JB):                8.716
Skew:                          -0.257   Prob(JB):                       0.0128
Kurtosis:                       3.661   Cond. No.                         38.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Mesmo depois da Transformação de Box-Cox os resíduos não seguem uma distribuição normal, incapacitando o modelo de realizar previsões mais acertivas

In [70]:
stat, p_value = shapiro(modelo_bc.resid)
stat , p_value

(0.9626880288124084, 6.149879254735424e-07)

### LogLike do Modelo ZINB ainda é melhor do que o do Modelo Box-Cox

In [72]:
llrtest(modelo_zinb, modelo_bc)

LogLike Modelo Ref: -557.26
LogLike Modelo: -803.49 
Estatistica Chi2: -492.45
P-value Chi2: 1.0
